In [ ]:
import getpass
import pandas as pd
import requests
import json

In [ ]:
username=input("NTS username: ")
password = getpass.getpass("NTS password: ")

# Haal token

In [ ]:
url = "https://terminologieserver.nl/auth/realms/nictiz/protocol/openid-connect/token"

payload='username='+username+'&password='+password+'&grant_type=password&client_id=cli_client'
headers = { 'Content-Type': 'application/x-www-form-urlencoded' }

response = requests.request("POST", url, headers=headers, data=payload)

token = response.json()
access_token = token["access_token"]

#print(access_token)

# Haal lijst van ValueSets op (op basis van probleem in titel)

In [ ]:
#url = "https://terminologieserver.nl/fhir/ValueSet?_count=40&name=Probleem"
url = "https://terminologieserver.nl/fhir/ValueSet?_count=4"

payload={}
headers = { 'Authorization': 'Bearer ' + access_token }

response = requests.request("GET", url, headers=headers, data=payload)

#print(response.text)
#print(json.dumps(response.json(), indent=2))

In [ ]:
data = response.json()

entries = []
for entry in data["entry"]:
    entries.append (entry["resource"]);

In [ ]:
df = pd.DataFrame(entries)

In [ ]:
df = df[['id','name']]
df['url'] = 'https://terminologieserver.nl/fhir/ValueSet/' + df['id']
df

In [ ]:
def putDhealthLab(vs, idx):
    print(idx, " dHealth PUT ", vs['id'], vs["name"], len(vs["compose"]["include"]))
    url = "https://dhealth.usor.nl:4003/hapi-fhir-jpaserver/fhir/ValueSet/" + vs['id']
    payload=json.dumps(vs, indent=2)
    headers = { 'Content-Type': 'application/json' }
    response = requests.request("PUT", url, headers=headers, data=payload)
    print(idx, "result: ", response.status_code)
    
def putSnowstorm(vs, idx):
    print(idx, "Snowstorm PUT ", vs['id'], vs["name"])
    url = "https://snowstorm.web.rug.nl/fhir/ValueSet/" + vs['id']
    payload=json.dumps(vs, indent=2)
    headers = { 'Content-Type': 'application/json' }
    response = requests.request("PUT", url, headers=headers, data=payload)
    print(idx, "result: ", response.status_code)
    

# Per ValueSet details ophalen en dan omzetten ontoserver R4 filters naar snowstorm STU3 filters

In [ ]:
# ProbleemAnatomischeLocatieCodelijst heeft een filter!

# TODO: Haal ook $expand vs op en put die in de dHealthLab versie
# Gebruik een filter om $expand,$lookup queries door te sturen naar snowstorm vanuit dHealthLab endpoint??

def doValueSet(vs, idx):
    # Only 2017 version
    if(vs["version"] == "2017-12-31T00:00:00"):
        if ("filter" in vs["compose"]["include"][0]):
            # adjust filter and copy to dHealth Snowstorm
            for filter in vs["compose"]["include"][0]["filter"]:
                # [{'property': 'concept', 'op': 'descendent-of', 'value': '442083009'}]
                # [{'property': 'concept', 'op': '=', 'value': '<<442083009'}]
                if (filter["op"] == "descendent-of"):
                    filter["op"] = "="
                    filter["value"] = "<<" + filter["value"]
            #print(vs["compose"]["include"][0]["filter"])
            #print(json.dumps(vs, indent=2))
            putSnowstorm(vs, idx)
        else:
            # copy to dHealth
            putDhealthLab(vs, idx)

# Loop over alle 2017 ValueSets en toon filter als die er is

idx = 0
for entry in data["entry"]:
    url = entry['fullUrl']
    payload={}
    headers = { 'Authorization': 'Bearer ' + access_token }
    response = requests.request("GET", url, headers=headers, data=payload)
    doValueSet (response.json(), idx)
    idx = idx + 1